<a href="https://colab.research.google.com/github/Shuraimi/fastai_DL_course/blob/main/Lesson5_Notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#hide
! [ -e /content ] && pip install -Uqq fastbook kaggle waterfallcharts treeinterpreter dtreeviz==1.4.1
import fastbook
fastbook.setup_book()

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
Mounted at /content/gdriv

In [ ]:
#hide
from fastbook import *
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

#Tabular Modelling Deep dive

Tabular modelling takes data in form of a table(spreadsheet, CSV,etc). The *objective* is to predict the value of one column based on values of other columns.

## Categorical embeddings

The numeric data can be directly fed to the model but we need to convert categorical variables (whose values correspond to different categories) into number before feeding them into the model.

*Continuous variables*- can be directly fed to the model and can also be added and multiplied whereas *categorical variables* which contain discrete number of levels and can't be multiplied or added.

>Entity embedding: not only reduces memory usage and speeds up NN, but more <mark>importantly by mapping similiar values close to each other in the embedding space 🌌 it reveals intrinsic properties of categorical variables.

**It is especially useful for datasets with high cardinality features.**

What is entity embedding used for?
>As entity embedding defines a distance measure for categorical variables, it can be used for visualising categorical data and for data clustering.

The embedding layer is just another layer and the embedding transforms categorical variables into inputs that are both continuous and meaningful.

The all possible values of a Categorical variable are called as levels/classes or categories.

We provide the model fundamentally categorical data about discrete entities and then model learns an embedding for these entities that defines a continuous notion of distance between them and these embeddings are continuous.

*Another benefit* is that we can combine the continuous embeddings with continuous input data by just concatenating them and then feeding it to the first dense layer of NN. i.e in other words, the raw categorical data is transformed by the embedding layer before it interacts with  raw continuous input data.

DL is not always the best staring oint for Tabular data but is best for unstructured data such as images, text, videos, etc.

##Beyond Deep learning

Recent researches show that majority of the datasets can be best modelled using only two methods:
1. Ensemble of DTs - Random Forest mainly used for structured data
2. Multilayered NN - mainly for unstructured data.

These two approaches give similar results to many kinds of structured data.

But ensemble of DTs:-
- train faster
- don't require GPU for inference
- easier to interpret
- require less hyperparameter tuning.

Interpreting a model of Tabular data is easier with ensemble of DTs. There are tools and methods for answering questions like :
1. Which columns in the dataset were the most important for your predictions? - getting the feature importances
2. How are they related to the dependent variable?
3. How do they interact with each other?
4. And which particular features were most important for some particular observation?

Therefore ensemble of DTs are out first approach for handling tabular data.

The exception to this guideline is when Dataset meets one of the conditions:
- there are some high cardinality variables that are very important such as zip code which has many possible levels.
- there are few columns that contain data that should be best understood by a NN.

But in practise when we encounter the datasets which meet these exceptions, we should try out both DTs and NNs to see which works best.

## The dataset

The dataset we use in this chapter is from the Blue Book for Bulldozers Kaggle competition, which has the following description: "The goal of the contest is to predict the sale price of a particular piece of heavy equipment at auction based on its usage, equipment type, and configuration.  The data is sourced from auction result postings and includes information on usage and equipment configurations."

In [ ]:
comp='bluebook-for-bulldozers'
path=URLs.path(comp)
path

Path('/root/.fastai/archive/bluebook-for-bulldozers')

In [ ]:
path=Path('/content/gdrive/MyDrive/bluebook-for-bulldozers')

In [ ]:
Path.BASE_PATH=path

In [ ]:
path.ls()

(#17) [Path('Train.7z'),Path('TrainAndValid.7z'),Path('Valid.7z'),Path('Machine_Appendix.csv'),Path('median_benchmark.csv'),Path('random_forest_benchmark_test.csv'),Path('Test.csv'),Path('test_predictions.csv'),Path('Train.csv'),Path('train_tmp.csv')...]

### Look at the data

First week start by reading data into Pandas DataFrame. Generally it's a good idea to specify `low_memory=False` unless Pandas runs out of memory and returns an error.

The `low_memory` parameter by default is set to True which tells Pandas to only look through a few rows at a time to decide what data type each column has. This means Pandas can actually end up using different data types for each rows which can lead to data processing or model training problems.

In [ ]:
df=pd.read_csv(path/'TrainAndValid.csv',low_memory=False)
df

,SalesID,SalePrice,MachineID,ModelID,...,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000.0,999089,3157,...,NaN,NaN,Standard,Conventional
1,1139248,57000.0,117657,77,...,NaN,NaN,Standard,Conventional
2,1139249,10000.0,434808,7009,...,NaN,NaN,NaN,NaN
3,1139251,38500.0,1026470,332,...,NaN,NaN,NaN,NaN
4,1139253,11000.0,1057373,17311,...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
412693,6333344,10000.0,1919201,21435,...,NaN,NaN,NaN,NaN
412694,6333345,10500.0,1882122,21436,...,NaN,NaN,NaN,NaN
412695,6333347,12500.0,1944213,21435,...,NaN,NaN,NaN,NaN
412696,6333348,10000.0,1794518,21435,...,NaN,NaN,NaN,NaN


In [ ]:
df.columns

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'saledate', 'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc',
       'fiModelSeries', 'fiModelDescriptor', 'ProductSize',
       'fiProductClassDesc', 'state', 'ProductGroup', 'ProductGroupDesc',
       'Drive_System', 'Enclosure', 'Forks', 'Pad_Type', 'Ride_Control',
       'Stick', 'Transmission', 'Turbocharged', 'Blade_Extension',
       'Blade_Width', 'Enclosure_Type', 'Engine_Horsepower', 'Hydraulics',
       'Pushblock', 'Ripper', 'Scarifier', 'Tip_Control', 'Tire_Size',
       'Coupler', 'Coupler_System', 'Grouser_Tracks', 'Hydraulics_Flow',
       'Track_Type', 'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb',
       'Pattern_Changer', 'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type',
       'Travel_Controls', 'Differential_Type', 'Steering_Controls'],
      dtype='object')

There are many columns, but later we'll see how to get the interesting columns.

Now is a good step to handle ordinal columns, i.e. columns which contain strings having natural ordering. Such as the `ProductSize` column.

In [ ]:
#get the unique columns in ProductSize column
df.ProductSize.unique()

array([nan, 'Medium', 'Small', 'Large / Medium', 'Mini', 'Large', 'Compact'], dtype=object)

In [ ]:

# now create a sizes variable with sizes in order
sizes='Large','Large / Medium','Medium','Small','Mini','Compact'

In [ ]:
sizes

('Large', 'Large / Medium', 'Medium', 'Small', 'Mini', 'Compact')

In [ ]:
# change dtype of ProductSize column
df.ProductSize=df.ProductSize.astype('category')
df.ProductSize=df.ProductSize.cat.set_categories(sizes,ordered=True)

In [ ]:
df.ProductSize.cat.codes

,0
0,-1
1,2
2,-1
3,3
4,-1
...,...
412693,4
412694,4
412695,4
412696,4


Since the metric for the competition is RMSE, we need to apply log to our dependent variable.

In [ ]:
dep_var='SalePrice'

In [ ]:
df[dep_var]=np.log(df[dep_var])

In [ ]:
df

,SalesID,SalePrice,MachineID,ModelID,...,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,11.097410,999089,3157,...,NaN,NaN,Standard,Conventional
1,1139248,10.950807,117657,77,...,NaN,NaN,Standard,Conventional
2,1139249,9.210340,434808,7009,...,NaN,NaN,NaN,NaN
3,1139251,10.558414,1026470,332,...,NaN,NaN,NaN,NaN
4,1139253,9.305651,1057373,17311,...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
412693,6333344,9.210340,1919201,21435,...,NaN,NaN,NaN,NaN
412694,6333345,9.259131,1882122,21436,...,NaN,NaN,NaN,NaN
412695,6333347,9.433484,1944213,21435,...,NaN,NaN,NaN,NaN
412696,6333348,9.210340,1794518,21435,...,NaN,NaN,NaN,NaN


## Decision trees

The decision tree is based on set of yes or no questions.

The basic steps to train a decision tree can be written down very easily:

1. Loop through each column of the dataset in turn.
1. For each column, loop through each possible level of that column in turn.
1. Try splitting the data into two groups, based on whether they are greater than or less than that value (or if it is a categorical variable, based on whether they are equal to or not equal to that level of that categorical variable).
1. Find the average sale price for each of those two groups, and see how close that is to the actual sale price of each of the items of equipment in that group. That is, treat this as a very simple "model" where our predictions are simply the average sale price of the item's group.
1. After looping through all of the columns and all the possible levels for each, pick the split point that gave the best predictions using that simple model.
1. We now have two different groups for our data, based on this selected split. Treat each of these as separate datasets, and find the best split for each by going back to step 1 for each group.
1. Continue this process recursively, until you have reached some stopping criterion for each group—for instance, stop splitting a group further when it has only 20 items in it.

### Handling dates

Handling data is first piece of data preparation. We handle dates so that our model can understand which is more recent and which is not.

We use fastai's function `add_datepart`.

In [ ]:
df=add_datepart(df,'saledate')

/usr/local/lib/python3.10/dist-packages/fastai/tabular/core.py:23: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[date_field] = pd.to_datetime(df[date_field], infer_datetime_format=True)


The same for test dataset

In [ ]:
df_test=pd.read_csv('/content/drive/MyDrive/bluebook-for-bulldozers/Test.csv',low_memory=False)
df_test=add_datepart(df_test,'saledate')

/usr/local/lib/python3.10/dist-packages/fastai/tabular/core.py:23: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[date_field] = pd.to_datetime(df[date_field], infer_datetime_format=True)


Now we check all the columns starting with sale

In [ ]:
' '.join(col for col in df.columns if col.startswith('sale'))

'saleYear saleMonth saleWeek saleDay saleDayofweek saleDayofyear saleIs_month_end saleIs_month_start saleIs_quarter_end saleIs_quarter_start saleIs_year_end saleIs_year_start saleElapsed'

To do a bit more cleaning, we'll use `TabularProc` and `TabularPandas`

### Using `TabularPandas` and `TabularProc`

This is the second piece of data prep is handling strings and missing values using fastai's `TabularPandas` and `TabularProc`.

`TabularPandas` is just like `Transform` which returns same object after modifying.

`Categorify` is a `TabularProc` which replaces a column with numeric categorical column.

`FilMissing` is a `TabularProc` which replaces every missing values with median of column and adds a new Boolean column which is set to True if it was empty.

These two transforms are needed for nearly every tabular dataset you will use, so this is a good starting point for your data processing.

In [ ]:
proc=[Categorify,FillMissing]

Now since we don't want random dev set, we use `np.where` to get indices of all items where given cond i.e. time period is True

In [ ]:
cond=(df.saleYear<2011)|(df.saleMonth<10)
train_idx=np.where( cond)[0]
valid_idx=np.where(~cond)[0]

In [ ]:
splits=(list(train_idx),list(valid_idx))

In [ ]:
len(splits[1])

7988

`TabularPandas`needs to know which columns are categorical and which are continuous. So we use helper function `cont_cat_split`

In [ ]:
cont,cat=cont_cat_split(df,1,dep_var=dep_var)

In [ ]:
help(cont_cat_split)

Help on function cont_cat_split in module fastai.tabular.core:

cont_cat_split(df, max_card=20, dep_var=None)
    Helper function that returns column names of cont and cat variables from given `df`.



In [ ]:
help(TabularPandas)

Help on class TabularPandas in module fastai.tabular.core:

class TabularPandas(Tabular)
 |  TabularPandas(df, procs=None, cat_names=None, cont_names=None, y_names=None, y_block=None, splits=None, do_setup=True, device=None, inplace=False, reduce_memory=True)
 |  
 |  A `Tabular` object with transforms
 |  
 |  Method resolution order:
 |      TabularPandas
 |      Tabular
 |      fastcore.foundation.CollBase
 |      fastcore.basics.GetAttr
 |      fastai.data.core.FilteredBase
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  transform(self, cols, f, all_col=True)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from Tabular:
 |  
 |  __init__(self, df, procs=None, cat_names=None, cont_names=None, y_names=None, y_block=None, splits=None, do_setup=True, device=None, inplace=False, reduce_memory=True)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  copy(self)
 |  
 |  decode(self)
 |  
 | 

In [ ]:
to=TabularPandas(df,proc,cat,cont,y_names=dep_var,splits=splits)

`TabularPandas` behaves much like fastai datasets and has `train` and `valid` attributes.

In [ ]:
len(to.train),len(to.valid)

(404710, 7988)

We can still see the column as strings and we use `show` to display few.

In [ ]:
to.show(3)

,UsageBand,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,state,ProductGroup,ProductGroupDesc,Drive_System,Enclosure,Forks,Pad_Type,Ride_Control,Stick,Transmission,Turbocharged,Blade_Extension,Blade_Width,Enclosure_Type,Engine_Horsepower,Hydraulics,Pushblock,Ripper,Scarifier,Tip_Control,Tire_Size,Coupler,Coupler_System,Grouser_Tracks,Hydraulics_Flow,Track_Type,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,saleIs_month_end,saleIs_month_start,saleIs_quarter_end,saleIs_quarter_start,saleIs_year_end,saleIs_year_start,auctioneerID_na,MachineHoursCurrentMeter_na,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,saleYear,saleMonth,saleWeek,saleDay,saleDayofweek,saleDayofyear,saleElapsed,SalePrice
0,Low,521D,521,D,#na#,#na#,#na#,Wheel Loader - 110.0 to 120.0 Horsepower,Alabama,WL,Wheel Loader,#na#,EROPS w AC,None or Unspecified,#na#,None or Unspecified,#na#,#na#,#na#,#na#,#na#,#na#,#na#,2 Valve,#na#,#na#,#na#,#na#,None or Unspecified,None or Unspecified,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,Standard,Conventional,False,False,False,False,False,False,False,False,1139246,999089,3157,121,3.0,2004,68.0,2006,11,46,16,3,320,1.163635e+09,11.097410
1,Low,950FII,950,F,II,#na#,Medium,Wheel Loader - 150.0 to 175.0 Horsepower,North Carolina,WL,Wheel Loader,#na#,EROPS w AC,None or Unspecified,#na#,None or Unspecified,#na#,#na#,#na#,#na#,#na#,#na#,#na#,2 Valve,#na#,#na#,#na#,#na#,23.5,None or Unspecified,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,Standard,Conventional,False,False,False,False,False,False,False,False,1139248,117657,77,121,3.0,1996,4640.0,2004,3,13,26,4,86,1.080259e+09,10.950807
2,High,226,226,#na#,#na#,#na#,#na#,Skid Steer Loader - 1351.0 to 1601.0 Lb Operating Capacity,New York,SSL,Skid Steer Loaders,#na#,OROPS,None or Unspecified,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,Auxiliary,#na#,#na#,#na#,#na#,#na#,None or Unspecified,None or Unspecified,None or Unspecified,Standard,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,False,False,False,False,False,False,False,False,1139249,434808,7009,121,3.0,2001,2838.0,2004,2,9,26,3,57,1.077754e+09,9.210340


To see underlying items.

In [ ]:
to.items.head(3)

,SalesID,SalePrice,MachineID,ModelID,...,saleIs_year_start,saleElapsed,auctioneerID_na,MachineHoursCurrentMeter_na
0,1139246,11.097410,999089,3157,...,1,1.163635e+09,1,1
1,1139248,10.950807,117657,77,...,1,1.080259e+09,1,1
2,1139249,9.210340,434808,7009,...,1,1.077754e+09,1,1


The conversion of categorical columns to numbers is done by simply replacing

To save the `TabularPandas` object, we use pickle.

In [ ]:
save_pickle('/content/drive/MyDrive/bluebook-for-bulldozers/to.pkl',to)